In [5]:
import base64
import requests


def run(fname, app_id, app_code):

    inputdata = open(fname,"rb").read()
    #print("inputdata.read:", inputdata)
    inputdata = base64.b64encode(inputdata)
    #print("b64encoded inputdata:", inputdata)

    url = "https://rme.api.here.com/2/matchroute.json"
    params = {
        "app_id": app_id,
        "app_code": app_code,
        "routemode": "car",
        "file": inputdata
    }

    result = requests.get(url=url, params=params)

    print("status code:", result.status_code)

    content = result.json()
    #print("content:", content)

    routes, traces, linkmap = extract_matched_info(content)

    print("routes:", routes)

    print("traces:")
    for t in traces: print(t)

    print("linkmap:")
    for k, v in linkmap.items(): print(k, v)

    return


def extract_matched_info(content):

    linkmap = {}
    routes = []
    for link in content["RouteLinks"]:
        linkId = link["linkId"]
        routes.append(linkId)
        length = float(link["linkLength"])
        fc = int(link["functionalClass"])
        shape = []
        shapes = link["shape"].strip().split(" ")
        nshapes = len(shapes)
        for i in range(0, nshapes, 2):
            shape.append((float(shapes[i]), float(shapes[i+1])))
        linkmap[linkId] = [fc, length, shape]

    traces = []
    for trace in content["TracePoints"]:
        lat = float(trace["lat"])
        lon = float(trace["lon"])
        linkIdMatched = trace["linkIdMatched"]
        latMatched = float(trace["latMatched"])
        lonMatched = float(trace["lonMatched"])
        matchOffsetOnLink = float(trace["matchOffsetOnLink"])
        matchDistance = float(trace["matchDistance"])
        traces.append((lat, lon, linkIdMatched, latMatched, lonMatched, matchOffsetOnLink, matchDistance))

    return routes, traces, linkmap


In [6]:
fname = "test001.gpx"

# obtain app_id, app_code from developer.here.com
app_id = "gtswh2sa822DHthznV1X"
app_code = "zd2FnIRxq25-BNHJSqgjtg"

run(fname, app_id, app_code)

status code: 200
routes: [25664478, 1022687683, 1022687684, 25664459, 781742854, 781742855, 25664450, 25664444, 25664440, 25664427, 860272608, 860272609, 781742870, 1022687692, 1022687693]
traces:
(51.10177, 0.39349, 25664478, 51.10178, 0.3935, 0.6290743846692971, 1.46)
(51.10181, 0.39335, 25664478, 51.10182, 0.39336, 0.8790743846674338, 1.46)
(51.10255, 0.39366, 781742855, 51.10255, 0.39367, 0.0, 1.11)
(51.10398, 0.39466, 860272609, 51.10398, 0.39468, 0.1535150210712758, 1.87)
(51.10501, 0.39533, 1022687693, 51.10501, 0.39532, 0.2847498087963734, 1.33)
linkmap:
25664478 [5, 43.08, [(51.10168, 0.39385), (51.10184, 0.39329)]]
1022687683 [4, 35.21, [(51.10184, 0.39329), (51.10214, 0.39345)]]
1022687684 [4, 11.66, [(51.10214, 0.39345), (51.10224, 0.3935)]]
25664459 [4, 9.33, [(51.10224, 0.3935), (51.10232, 0.39354)]]
781742854 [4, 27.16, [(51.10232, 0.39354), (51.10245, 0.39361), (51.10255, 0.39367)]]
781742855 [4, 16.8, [(51.10255, 0.39367), (51.10269, 0.39376)]]
25664450 [4, 44.74, [(51